Extract Depart and Delay Data for Delta Flights from Atl, LA, CHI, DAL (FT Worth), Den, NY (JFK), SF, Seattle (Tacona), McCarran (LV) airports for 2019.

Data File Path: r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'

Final Pickle: r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/bts_10_airport_departures2019.pickle'

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os

In [2]:
import pandas as pd
import pickle

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/usr/local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
def extract_usairports(selection=None, download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'):
    
    file_path = os.path.join(download_path,f'{selection} airports.csv')
    
    if not os.path.isfile(file_path):
    
        usairports_url = "https://en.wikipedia.org/wiki/List_of_airports_in_the_United_States"

        driver = webdriver.Chrome(chromedriver)
        driver.get(usairports_url)

        driver.execute_script(("window.scrollBy(0,700)"))

        xpath = "//*[@id='mw-content-text']/div[1]/table[2]/thead/tr/th[7]"

        sort_enplanement = driver.find_element_by_xpath(xpath)

        for click in range(2):

            sort_enplanement.click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        driver.close()

        airport_data_list = []

        airport_table = soup.find('table',class_ = 'wikitable sortable jquery-tablesorter')

        headers = airport_table.find_all('th')
        airport_columns = [header.text.replace("\n", "") for header in headers]

        airports = airport_table.find_all('tr')[1:selection + 1]

        for airport in airports:

            td = airport.find_all('td')
            airport_data_list.append([data.text.replace("\n", "") for data in td])

        airport_df = pd.DataFrame(airport_data_list, columns = airport_columns)

        airport_df.to_csv(file_path)
    
    return file_path

In [9]:
def download_bts_data(selection, year, headless=True, download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'):
    
    option = webdriver.ChromeOptions()
    
    #if headless:
        #option.add_argument('headless')
    
    if download_path is not None:
    
        prefs = {}
        os.makedirs(download_path, exist_ok=True)
        prefs["profile.default_content_settings.popups"]=0
        prefs["download.default_directory"]=download_path
        option.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(chromedriver, options = option)
    
    bts_website = "https://transtats.bts.gov/ONTIME/Departures.aspx"
    driver.get(bts_website)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    airports = [airport.get('value') for airport in soup.find_all('option')]    
    
    selected_airports_df = pd.read_csv(extract_usairports(selection))
    
    time.sleep(10)
    
    airport_FAA_names = list(selected_airports_df['FAA'])
       
    time.sleep(10)
    
    for i, airport in enumerate(airport_FAA_names):
        
        file_path = os.path.join(download_path,f'{airport}{year}.csv')
            
        try:
            if airport in airports and i == 0:

                if os.path.isfile(file_path):
                    os.remove(file_path)

                select_airport = driver.find_element_by_xpath(f"//select[@id='cboAirport']/option[@value='{airport}']")
                select_airport.click()

                airline = 'DL' #Requires shortform name of airline
                select_airline = driver.find_element_by_xpath(f"//select[@id='cboAirline']/option[@value='{airline}']")
                select_airline.click()

                time.sleep(3)

                required_fields = ['Statistics','Months','Days', year]

                for field in required_fields:

                    if field == year:

                        check_box = driver.find_element_by_xpath(f"//input[@type='checkbox' and @value={year}]")
                        check_box.click()

                        time.sleep(3)

                    else:   

                        check_box = driver.find_element_by_xpath(f"//input[@id='chkAll{field}']")
                        check_box.click()
                        
                        time.sleep(3)

                submit = driver.find_element_by_xpath(f"//input[@name='btnSubmit' and @value='Submit']")
                submit.click()

                time.sleep(15)

                download = driver.find_element_by_xpath(f"//a[@id='DL_CSV']")
                download.click()

                time.sleep(30)

                bts_df = pd.read_csv(os.path.join(download_path,'Detailed_Statistics_Departures.csv'), header = 6, skipfooter= 1, engine = 'python')
                bts_df['Airport'] = airport

                os.remove(os.path.join(download_path,'Detailed_Statistics_Departures.csv')) 

                bts_df.to_csv(file_path)

                yield file_path

            elif airport in airports and i != 0:

                if not os.path.isfile(file_path):

                    select_airport = driver.find_element_by_xpath(f"//select[@id='cboAirport']/option[@value='{airport}']")
                    select_airport.click()

                    submit = driver.find_element_by_xpath(f"//input[@name='btnSubmit' and @value='Submit']")
                    submit.click()

                    time.sleep(10)

                    download = driver.find_element_by_xpath(f"//a[@id='DL_CSV']")
                    download.click()

                    time.sleep(10)

                    bts_df = pd.read_csv(os.path.join(download_path,'Detailed_Statistics_Departures.csv'), header = 6, skipfooter= 1, engine = 'python')
                    bts_df['Airport'] = airport

                    os.remove(os.path.join(download_path,'Detailed_Statistics_Departures.csv')) 

                    file_path = os.path.join(download_path,f'{airport}{year}.csv')

                    bts_df.to_csv(file_path)

                    yield file_path

                else:

                    yield file_path

            else:

                print(f'{airport} not in BTS database')

        except:

            print('driver error')

            continue
        
    driver.close()

In [10]:
def execute_download_bts_data(selection, year, **kwargs):
    
    gen = download_bts_data(selection, year)
    
    file_list = []
    
    for download in range(selection + 1):
        
        try: 
            file_list.append(next(gen))
        
        except:
            
            continue
            
    return file_list

In [11]:
def create_bts_pickle(selection, year, download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data', **kwargs):
    
    file_path = os.path.join(download_path,f'raw_bts_{selection}_airport_departures{year}.pickle')
    
    if os.path.isfile(file_path):
        
        pass
    
    else:
        
        airport_path_list = execute_download_bts_data(selection, year)

        raw_bts_df = pd.concat([pd.read_csv(airport_path).iloc[:, 1:] for airport_path in airport_path_list], axis = 0)

        with open(file_path, 'wb') as to_write:

            pickle.dump(raw_bts_df, to_write)
    
    return file_path

In [12]:
create_bts_pickle(25, '2019')

'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/raw_bts_25_airport_departures2019.pickle'